In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2020-12-06 18:29:24,004 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-06 18:29:24,005 - settings - Setting database.user to celiib
INFO - 2020-12-06 18:29:24,006 - settings - Setting database.password to newceliipass
INFO - 2020-12-06 18:29:24,010 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-06 18:29:24,010 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-06 18:29:24,021 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-06 18:29:24,221 - settings - Setting database.host to at-database.ad.bcm.edu


Sleeping 12 sec before conneting
Done sleeping


INFO - 2020-12-06 18:29:24,222 - settings - Setting database.user to celiib
INFO - 2020-12-06 18:29:24,223 - settings - Setting database.password to newceliipass
INFO - 2020-12-06 18:29:24,224 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-06 18:29:24,401 - settings - Setting enable_python_native_blobs to True


Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 7394


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

# Segment_id we are currently checking

In [6]:
segment_id=864691136535575714

# Checking the Table

In [7]:
#minnie.Decomposition()

In [8]:
minnie.Decimation() & dict(segment_id=segment_id)

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691136535575714,0,0.10,314807,622049,=BLOB=
864691136535575714,0,0.25,781389,1555047,=BLOB=


In [9]:
import neuron_visualizations as nviz
du.plot_decimated_mesh_with_somas(segment_id)

Segment_id = 864691136535575714
vertices = 781389, faces= = 1555047


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Debugging the neuron creation

In [10]:
#1) Get the segment id from the key
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()



----- Working on 864691136535575714-------
somas = [[<trimesh.Trimesh(vertices.shape=(23444, 3), faces.shape=(46516, 3))>], array([641.1727]), array([0.849])]


In [14]:
import preprocessing_vp2 as pre
import soma_extraction_utils as sm
import skeleton_utils as sk

import neuron
tu = reload(tu)
du = reload(du)
sm = reload(sm)
pre = reload(pre)
import compartment_utils as cu
cu = reload(cu)
sk= reload(sk)

neuron = reload(neuron)
print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=True,
widths_to_calculate=["no_spine_median_mesh_center"]

        )


--- Beginning preprocessing of 864691136535575714---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(23444, 3), faces.shape=(46516, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(23444, 3), faces.shape=(46516, 3))>]
soma_mesh_list_centers = [array([1329540.13862822,  476301.83616277,  701847.28207217])]
xvfb-run -n 7394 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging_Graph/temp/neuron_1386.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging_Graph/temp/neuron_1386_remove_interior.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging_Graph/temp/remove_interior_235217.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging_Graph/temp/neuron_1386.off
removed temporary output file: /

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:16<00:00, 16.06s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 19.855876922607422
connecting at the root
branches_touching_root = [105]
length of Graph = 28055
Working on path [23222. 23230. 23241. 23242.]
path_degrees = [3, 2, 2, 3]
Working on path [28055. 23229. 23237. 23246. 23249.]
path_degrees = [4, 2, 2, 2, 3]
Working on path [28056. 23256. 23266. 23270. 23277. 23284.]
path_degrees = [5, 2, 2, 2, 2, 3]
Working on path [27721. 27736. 27748. 27751.]
path_degrees = [3, 2, 2, 3]
max(kept_branches_idx) = 105, len(kept_branches_idx) = 102
empty_indices % = 0.0
 conflict_indices % = 0.006570452720678363



AFTER face_lookup_resolved_test



Decomposing first pass: 30.39211893081665
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Found len(mesh_large_idx) MAP candidates: [11749, 15605, 14004, 972, 5689, 1594, 227, 7343, 3341, 5108, 369, 753, 7436, 14549, 15168, 4011, 14537, 400, 1174, 4899, 1012, 6927, 11192, 1136, 427, 10913, 14815, 37643, 16011, 8288, 15167, 13944, 3788, 7117, 1855, 8488, 14250, 1361, 4219, 33460, 22139, 43018, 14031, 19861, 2565, 1038, 19881, 26944, 2099, 9588, 19213, 9915, 6659, 5014, 3667]
mesh_large_connectivity: 2.5424916744232178
Finding MAP candidates connected components: 0.0005695819854736328
len(filtered_pieces) = 2
skeleton_connectivity_MP : 0.646815299987793
Grouping MP Sublimbs by Graph: 0.15948820114135742
Divinding into MP and MAP pieces: 9.5367431640625e-07
--- Working on MAP piece 0---
MAP Filtering Soma Pieces: 0.04044675827026367
inside skeletonize_connected_branch and use_surface_after_CGAL=False, surface_reconstruction_size=1000
     Starting S

KeyboardInterrupt: 

In [36]:
tu = reload(tu)
tu.face_neighbors_by_vertices_seperate(mesh,ex_faces)

[array([ 2,  3,  4,  7,  9, 10, 11, 12, 15, 16, 18, 19]),
 array([  7,   8,   9,  10,  11,  14,  15,  16,  23,  24,  25, 202]),
 array([  3,   4,  10,  11,  12,  16,  17,  18,  19,  25,  26, 200, 201,
        225, 251, 252]),
 array([ 27,  28,  29, 125, 126, 127, 128, 129, 130, 131, 132])]

In [34]:
ex_faces = [10,15,18,29]
mesh = current_neuron_mesh

f_verts = mesh.faces[ex_faces]
[np.unique(k[k!=-1]) for k in mesh.vertex_faces[f_verts]]

[array([ 2,  3,  4,  7,  9, 10, 11, 12, 15, 16, 18, 19]),
 array([  7,   8,   9,  10,  11,  14,  15,  16,  23,  24,  25, 202]),
 array([  3,   4,  10,  11,  12,  16,  17,  18,  19,  25,  26, 200, 201,
        225, 251, 252]),
 array([ 27,  28,  29, 125, 126, 127, 128, 129, 130, 131, 132])]

In [33]:
for k in mesh.vertex_faces[f_verts]:
    print(k)
    print(k[k!=-1] )
    print("---")

[[10  9  7  3  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [19 18 12 11 10  4  3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [16 15 11 10  9 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
[10  9  7  3  2 19 18 12 11 10  4  3 16 15 11 10  9]
---
[[ 15  14   9   8   7  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]
 [ 16  15  11  10   9  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]
 [202  25  24  23  16  15  14  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]]
[ 15  14   9   8   7  16  15  11  10   9 202  25  24  23  16  15  14]
---
[[ 26  25  18  17  16  11  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]
 [ 19  18  12  11  10   4   3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]
 [252 251 225 201 200  19  18  17  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1]]
[ 26  25  18  17  16  11  19  18  12  11  10   4   3 252 251 225 201

In [21]:
x = np.array([[1,3,4],[1,2],[1,4,6,9]])

In [22]:
[ex_numb[xi] for xi in x]

[array([9, 8, 8]), array([9, 5]), array([9, 8, 6, 6])]